# LangGraph quickstart
- Doc: https://langchain-ai.github.io/langgraph/agents/agents/

In [ ]:
from dotenv import load_dotenv

# API-KEY 읽어오기
load_dotenv()

## 2. Create an agent

In [ ]:
from langgraph.prebuilt import create_react_agent

# 도시의 날씨 정보를 반환하는 함수
def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"{city} 날씨는 항상 화창합니다!"

# ReAct 에이전트 생성
agent = create_react_agent(
    model="google_genai:gemini-2.5-flash",  # 사용할 LLM 모델 지정
    tools=[get_weather],                    # 에이전트가 사용할 도구 목록
    prompt="You are a helpful assistant"    # 에이전트의 역할과 동작을 정의하는 프롬프트
)

# 에이전트 실행
result = agent.invoke(
    {"messages": [{"role": "user", "content": "서울 날씨 어때?"}]},  # 사용자 메시지 입력
)
# 결과 출력
result

In [ ]:
# 각 단계의 메시지 출력
for message in result['messages']:
    message.pretty_print()

In [ ]:
# 에이전트 실행
result = agent.invoke(
    {"messages": [{"role": "user", "content": "오눌 주요 뉴스가 뭐야?"}]},  # 사용자 메시지 입력
)
# 결과 출력
result

## 3. Configure an LLM

In [ ]:
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent

# 채팅 모델 초기화
model = init_chat_model(
    model="google_genai:gemini-2.5-flash",  # 사용할 LLM 모델 지정
    temperature=0                           # 출력의 다양성 조절
                                            # 낮은 Temperature: 확률 분포를 더 날카롭게 만듬, 예측 가능하고 일관된 출력 생성
                                            # 높은 Temperature: 확률 분포를 더 평평하게 만듬, 창의적이고 다양한 출력 생성
)

# ReAct 에이전트 생성
agent = create_react_agent(
    model=model,          # 사용할 LLM 모델 객체 지정
    tools=[get_weather],  # 에이전트가 사용할 도구 목록
)

In [ ]:
# 에이전트 실행
result = agent.invoke(
    {"messages": [{"role": "user", "content": "서울 날씨 어때?"}]},  # 사용자 메시지 입력
)
# 각 단계의 메시지 출력
for message in result['messages']:
    message.pretty_print()

## 4. Add a custom prompt

### Static prompt

In [ ]:
from langgraph.prebuilt import create_react_agent

# ReAct 에이전트 생성
agent = create_react_agent(
    model="google_genai:gemini-2.5-flash",              # 사용할 LLM 모델 지정
    tools=[get_weather],                                # 에이전트가 사용할 도구 목록
    prompt="Never answer questions about the weather."  # 에이전트의 역할과 동작을 정의하는 정적 프롬프트
)

# 에이전트 실행
result = agent.invoke(
    {"messages": [{"role": "user", "content": "서울 날씨 어때?"}]}
)
# 각 단계의 메시지 출력
for message in result['messages']:
    message.pretty_print()

### Dynamic prompt

In [ ]:
from langchain_core.messages import AnyMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.prebuilt import create_react_agent

# 에이전트의 프롬프트 생성 함수
def prompt(state: AgentState, config: RunnableConfig) -> list[AnyMessage]:  
    # config에서 사용자 이름을 추출
    user_name = config["configurable"].get("user_name")
    # 시스템 메시지 생성: 사용자 이름을 포함
    system_msg = f"You are a helpful assistant. Address the user as {user_name}."
    # 시스템 메시지와 기존 상태의 메시지를 결합하여 반환
    return [{"role": "system", "content": system_msg}] + state["messages"]

# ReAct 에이전트 생성
agent = create_react_agent(
    model="google_genai:gemini-2.5-flash",  # 사용할 LLM 모델 지정
    tools=[get_weather],                    # 에이전트가 사용할 도구 목록
    prompt=prompt                           # 함수를 이용한 동적 프롬프트 생성
)

# 에이전트 실행
result = agent.invoke(
    {"messages": [{"role": "user", "content":  "서울 날씨 어때?"}]},
    # 에이전트 실행을 위한 설정 정보
    config={"configurable": {"user_name": "홍길동"}}
)
# 각 단계의 메시지 출력
for message in result['messages']:
    message.pretty_print()

## 5. Add memory

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

# 에이전트의 상태를 메모리에 저장 (프로그램이 종료되면 사라짐)
checkpointer = InMemorySaver()

# ReAct 에이전트 생성
agent = create_react_agent(
    model="google_genai:gemini-2.5-flash",  # 사용할 LLM 모델 지정
    tools=[get_weather],                    # 에이전트가 사용할 도구 목록
    checkpointer=checkpointer               # 에이전트의 상태를 저장하는 기능 추가
)

# 에이전트 실행을 위한 설정 정보
# 'thread_id'는 대화의 고유 식별자 역할
config = {"configurable": {"thread_id": "1"}}

# 에이전트 실행 (서울)
seoul_response = agent.invoke(
    {"messages": [{"role": "user", "content": "서울 날쌔 어때?"}]},
    config  
)
# 에이전트 실행 (부산)
pusan_response = agent.invoke(
    {"messages": [{"role": "user", "content": "부산 날씨 어때?"}]},
    config
)

In [ ]:
# 각 단계의 메시지 출력
for message in seoul_response['messages']:
    message.pretty_print()

In [ ]:
# 각 단계의 메시지 출력
for message in pusan_response['messages']:
    message.pretty_print()

## 6. Configure structured output

In [ ]:
from pydantic import BaseModel
from langgraph.prebuilt import create_react_agent

# 날씨 응답을 구조화하기 위한 모델
class WeatherResponse(BaseModel):
    conditions: str

# ReAct 에이전트 생성
agent = create_react_agent(
    model="google_genai:gemini-2.5-flash",  # 사용할 LLM 모델 지정
    tools=[get_weather],                    # 에이전트가 사용할 도구 목록
    response_format=WeatherResponse         # 응답을 WeatherResponse 형식으로 구조화
)

# 에이전트 실행
response = agent.invoke(
    {"messages": [{"role": "user", "content":  "서울 날씨 어때?"}]}
)

# 구조화된 응답 추출
response["structured_response"]

In [ ]:
# 각 단계의 메시지 출력
for message in response['messages']:
    message.pretty_print()

In [ ]:
from typing import Literal, Annotated
from pydantic import BaseModel
from langgraph.prebuilt import create_react_agent

# 날씨 응답을 구조화하기 위한 모델
class WeatherResponse(BaseModel):
    conditions: Annotated[
        # 허용되는 문자열 값들
        Literal["맑음", "흐림", "비", "눈"],
        # 필드에 대한 설명 (LLM이 이 정보를 사용해서 응답을 구조화함)
        "Includes only concise and essential current weather conditions."
    ]

# ReAct 에이전트 생성
agent = create_react_agent(
    model="google_genai:gemini-2.5-flash",  # 사용할 LLM 모델 지정
    tools=[get_weather],                    # 에이전트가 사용할 도구 목록
    response_format=WeatherResponse         # 응답을 WeatherResponse 형식으로 구조화
)

# 에이전트 실행
response = agent.invoke(
    {"messages": [{"role": "user", "content":  "서울 날씨 어때?"}]}
)

# 구조화된 응답 추출
response["structured_response"]

In [ ]:
# 각 단계의 메시지 출력
for message in response['messages']:
    message.pretty_print()